In [1]:
#Importando Bibliotecas
import pandas as pd
import numpy as np
import os
import datetime
from datetime import timedelta
from pathlib import Path

In [2]:
#Selecionando as Tábuas e Variáveis
tabua_mortalidade = 'AMERICAN EXPERIENCE'
tabua_mortalidade_inv = 'RRB-44'
tabua_entrada_inv = 'ALVARO VINDAS'
tabua_morbidez = 'Atuários Ingleses'
ano_dt_base = 2020 #Data base
tx_juros = 0.06 #Taxa de Juros
cres_sal = 0.01 #Crescimento Salarial
cres_benef = 0.01 #Crescimento do Benefício
wx = 0.01 #Rotatividade

In [3]:
#Importando as bases
bd_ativos = pd.read_csv(str(Path(os.getcwd())) + '\\1.SERVIDORES.csv', encoding='UTF-8', sep=';')#, thousands=',')

#Convertendo os dados para float
bd_ativos['DT_NASC_SERVIDOR'] = pd.to_datetime(bd_ativos['DT_NASC_SERVIDOR'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['DT_ING_ENTE'] = pd.to_datetime(bd_ativos['DT_ING_ENTE'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['DT_ING_CARGO'] = pd.to_datetime(bd_ativos['DT_ING_ENTE'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['VL_BASE_CALCULO'] = bd_ativos['VL_BASE_CALCULO'].str.replace(',', '.').astype(float)
bd_ativos['DT_NASC_CONJUGE'] = pd.to_datetime(bd_ativos['DT_NASC_CONJUGE'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['DT_NASC_NOVO'] = pd.to_datetime(bd_ativos['DT_NASC_NOVO'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['DT_NASC_INV'] = pd.to_datetime(bd_ativos['DT_NASC_INV'], yearfirst=True, format='%d/%m/%Y')

bd_ativos['DT_BASE'] = '31/12/2020'
bd_ativos['DT_BASE'] = pd.to_datetime(bd_ativos.DT_BASE, yearfirst=True, format='%d/%m/%Y')

In [4]:
#Calculando as Estatísticas dos Ativos
statis_ativos = pd.DataFrame()

statis_ativos['IDADE_SERVIDOR'] = round((bd_ativos.DT_BASE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365.25) - 0.5).astype(int) #Data-Base - Data de Nascimento do Servidor
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_ADM'] = round((bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365.25) - 0.5).astype(int) #Data de Ingresso no Ente - Data de Nascimento do Servidor
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_CONJUGE'] = bd_ativos.DT_NASC_CONJUGE.mask(bd_ativos.DT_NASC_CONJUGE.isnull(), '1800-01-01', axis=0)#, inplace=True) #Preenchendo as linhas vazias com alguma data
statis_ativos.IDADE_CONJUGE = round((bd_ativos.DT_BASE - statis_ativos.IDADE_CONJUGE) / timedelta(days=365.25) - 0.5).astype(int) #Calculando as Idades dos Cônjuges | Data-Base - Data de Nascimento do Cônjuge
statis_ativos.loc[(bd_ativos.CO_EST_CIVIL_SERVIDOR == 1) | (bd_ativos.CO_EST_CIVIL_SERVIDOR == 3) | (bd_ativos.CO_EST_CIVIL_SERVIDOR == 4) | (bd_ativos.CO_EST_CIVIL_SERVIDOR == 5) | (statis_ativos.IDADE_CONJUGE > 127), 'IDADE_CONJUGE'] = None #Removendo as datas inseridas
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_EMT'] = round(statis_ativos.IDADE_ADM - (bd_ativos['NU_TEMPO_RGPS']/365.25)) #Calculando a Idade de Entrada no Mercado de Trabalho | Idade de Admissão - (TSANT/365.25)
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_PROJ_APOS'] = bd_ativos.DT_PROV_APOSENT
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['DIF_ETARIA'] = statis_ativos.IDADE_CONJUGE - statis_ativos.IDADE_SERVIDOR #Calculando a Diferença Etária do Cônjuge | Idade do Cônjuge - Idade do Servidor
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_NOVO'] = bd_ativos.DT_NASC_NOVO.mask(bd_ativos.DT_NASC_NOVO.isnull(), '1800-01-01', axis=0)#, inplace=True) #Preenchendo as linhas vazias com alguma data
statis_ativos.IDADE_NOVO = round((bd_ativos.DT_BASE - statis_ativos.IDADE_NOVO) / timedelta(days=365.25) - 0.5).astype(int) #Calculando as Idades do Filho Mais Novo | Data-Base - Data de Nascimento do Filho Mais Novo
statis_ativos.loc[statis_ativos.IDADE_NOVO > 21, 'IDADE_NOVO'] = None #Removendo as datas inseridas
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_INV'] = bd_ativos.DT_NASC_INV.mask(bd_ativos.DT_NASC_INV.isnull(), '1800-01-01', axis=0)#, inplace=True) #Preenchendo as linhas vazias com alguma data
statis_ativos.IDADE_INV = round((bd_ativos.DT_BASE - statis_ativos.IDADE_INV) / timedelta(days=365.25) - 0.5).astype(int) #Calculando as Idades do Filho Inválido | Data-Base - Data de Nascimento do Filho Inválido
statis_ativos.loc[statis_ativos.IDADE_INV > 127, 'IDADE_INV'] = None #Removendo as datas inseridas
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['TP_PREFEITURA'] = statis_ativos.IDADE_SERVIDOR - statis_ativos.IDADE_ADM #Calculando o Tempo de Prefeitura | Idade do Servidor - Idade de Admissão
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['DIFER(r-x)'] = bd_ativos.DT_PROV_APOSENT - statis_ativos.IDADE_SERVIDOR #Calculando o Diferimento até a Aposentadoria | Idade Provável de Aposentadoria - Idade do Servidor
statis_ativos['DIFER(r-u)'] = bd_ativos.DT_PROV_APOSENT - statis_ativos.IDADE_ADM #Calculando o Diferimento até a Aposentadoria | Idade Provável de Aposentadoria - Idade de Admissão do Servidor
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['BENEF_PROJ'] = round(bd_ativos.VL_BASE_CALCULO * (1 + cres_sal)**statis_ativos['DIFER(r-x)'], 2)

#Idade de entrada no plano ou RPPS
#Perguntar a rúniu sobre a IDADE_EMT se considera o tempo do RPPS também

statis_ativos#.head(20)

,IDADE_SERVIDOR,IDADE_ADM,IDADE_CONJUGE,IDADE_EMT,IDADE_PROJ_APOS,DIF_ETARIA,IDADE_NOVO,IDADE_INV,TP_PREFEITURA,DIFER(r-x),DIFER(r-u),BENEF_PROJ
0,40,20,NaN,20.0,60,NaN,16.0,NaN,20,20,40,1342.21
1,45,42,48.0,25.0,62,3.0,NaN,NaN,3,17,20,1302.73
2,48,27,NaN,25.0,62,NaN,18.0,NaN,21,14,35,1964.27
3,31,23,NaN,23.0,62,NaN,NaN,NaN,8,31,39,4599.23
4,42,35,NaN,25.0,65,NaN,NaN,NaN,7,23,30,1382.88
...,...,...,...,...,...,...,...,...,...,...,...,...
357,45,22,NaN,22.0,57,NaN,NaN,NaN,23,12,35,2716.30
358,26,24,NaN,24.0,57,NaN,NaN,NaN,2,31,33,4046.86
359,36,25,NaN,25.0,60,NaN,NaN,NaN,11,24,35,4223.40
360,44,30,NaN,25.0,60,NaN,NaN,NaN,14,16,30,2511.36


In [5]:
#Projeção do Crescimento Salarial (Benefício de Aposentadoria Projetado)
proj_cres_sal = []
for l in range(len(bd_ativos)):
    linha = []
    for c in range(statis_ativos['DIFER(r-x)'][l]+1):
        benef_proj = round(bd_ativos.VL_BASE_CALCULO.loc[l] * ((1 + cres_sal)**c), 2)
        linha.append(benef_proj)
    proj_cres_sal.append(linha)

df_benef_proj = pd.DataFrame(data=proj_cres_sal, index=bd_ativos.ID_SERVIDOR_MATRICULA)

In [11]:
print(df_benef_proj)

                            0        1        2        3        4        5   \
ID_SERVIDOR_MATRICULA                                                         
11                     1100.00  1111.00  1122.11  1133.33  1144.66  1156.11   
2131                   1100.00  1111.00  1122.11  1133.33  1144.66  1156.11   
13                     1708.84  1725.93  1743.19  1760.62  1778.23  1796.01   
2072                   3378.49  3412.27  3446.40  3480.86  3515.67  3550.83   
2080                   1100.00  1111.00  1122.11  1133.33  1144.66  1156.11   
...                        ...      ...      ...      ...      ...      ...   
49                     2410.58  2434.69  2459.03  2483.62  2508.46  2533.54   
2883                   2972.73  3002.46  3032.48  3062.81  3093.43  3124.37   
647                    3326.21  3359.47  3393.07  3427.00  3461.27  3495.88   
230                    2141.74  2163.16  2184.79  2206.64  2228.70  2250.99   
757                    3426.86      NaN      NaN    

In [6]:
#Importando as Funções Biométricas
qx = pd.read_excel(str(Path(os.getcwd())) + '\\Tábuas\\Funções Biométricas (Com Fórmulas).xlsx', sheet_name='qx', usecols=[tabua_mortalidade])
px = pd.read_excel(str(Path(os.getcwd())) + '\\Tábuas\\Funções Biométricas (Com Fórmulas).xlsx', sheet_name='px', usecols=[tabua_mortalidade])
lx = pd.read_excel(str(Path(os.getcwd())) + '\\Tábuas\\Funções Biométricas (Com Fórmulas).xlsx', sheet_name='lx', usecols=[tabua_mortalidade])
dx = pd.read_excel(str(Path(os.getcwd())) + '\\Tábuas\\Funções Biométricas (Com Fórmulas).xlsx', sheet_name='dx', usecols=[tabua_mortalidade])
ix = pd.read_excel(str(Path(os.getcwd())) + '\\Tábuas\\Funções Biométricas (Com Fórmulas).xlsx', sheet_name='qx', usecols=[tabua_entrada_inv])

In [7]:
#Calculando as Funções Biométricas e Comutações
df_func_biometricas = pd.DataFrame()

df_func_biometricas['qx'] = qx
df_func_biometricas['px'] = px
df_func_biometricas['lx'] = lx
df_func_biometricas['dx'] = dx
df_func_biometricas['wx'] = wx #Rotatividade
df_func_biometricas['ix'] = ix #Entrada em Invalidez
df_func_biometricas['qx_m'] = df_func_biometricas.qx * ((1 - 0.5 * df_func_biometricas.ix) + (1 - 0.5 * df_func_biometricas.wx)) #Ambiente Multidecremental.
df_func_biometricas['ix_m'] = df_func_biometricas.ix * ((1 - 0.5 * df_func_biometricas.qx) + (1 - 0.5 * df_func_biometricas.wx)) #Ambiente Multidecremental.
df_func_biometricas['wx_m'] = df_func_biometricas.wx * ((1 - 0.5 * df_func_biometricas.qx) + (1 - 0.5 * df_func_biometricas.ix)) #Ambiente Multidecremental.
df_func_biometricas['qx_t'] = df_func_biometricas.qx_m + df_func_biometricas.ix_m + df_func_biometricas.wx_m 
df_func_biometricas['px_t'] = 1 - df_func_biometricas.qx_t
df_func_biometricas['v'] = 1
df_func_biometricas['Nx'] = 1
df_func_biometricas['lx_t'] = 1
for i in range(len(df_func_biometricas)): df_func_biometricas['v'].loc[i] =  1 / (1 + tx_juros)**i
df_func_biometricas['Dx'] = df_func_biometricas.lx * df_func_biometricas.v
for i in range(len(df_func_biometricas)): 
    df_func_biometricas.Nx.loc[i] = df_func_biometricas.Dx.loc[i:].sum()
    if i >= 1:
        df_func_biometricas['lx_t'].loc[i] = df_func_biometricas['lx_t'].loc[i - 1] * df_func_biometricas.px_t[i - 1]
    else:
        df_func_biometricas['lx_t'].loc[i] = 100000

df_func_biometricas.head()

C:\Users\Italo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,qx,px,lx,dx,wx,ix,qx_m,ix_m,wx_m,qx_t,px_t,v,Nx,lx_t,Dx
0,0.15470,0.84530,100000.0000,15470.000000,0.01,0.0,0.308627,0.0,0.019227,0.327853,0.672147,1.000000,1.206043e+06,100000.000000,100000.000000
1,0.06349,0.93651,84530.0000,5366.809700,0.01,0.0,0.126663,0.0,0.019683,0.146345,0.853655,0.943396,1.106043e+06,67214.700000,79745.283019
2,0.03550,0.96450,79163.1903,2810.293256,0.01,0.0,0.070822,0.0,0.019822,0.090645,0.909355,0.889996,1.026298e+06,57378.158007,70454.957547
3,0.02391,0.97609,76352.8970,1825.597767,0.01,0.0,0.047700,0.0,0.019880,0.067581,0.932419,0.839619,9.558431e+05,52177.114874,64107.364637
4,0.01770,0.98230,74527.2992,1319.133196,0.01,0.0,0.035312,0.0,0.019911,0.055223,0.944777,0.792094,8.917357e+05,48650.938492,59032.601435


In [8]:
df_anuidades = pd.DataFrame()
df_comp_VP_ativos = pd.DataFrame()

df_comp_VP_ativos['Br'] = statis_ativos.BENEF_PROJ
df_comp_VP_ativos['r-upu_t'] = statis_ativos.IDADE_PROJ_APOS.map(df_func_biometricas.lx_t) / statis_ativos.IDADE_ADM.map(df_func_biometricas.lx_t)
df_comp_VP_ativos['v_r-u'] = statis_ativos['DIFER(r-u)'].map(df_func_biometricas.v)
df_comp_VP_ativos['ar'] = bd_ativos.DT_PROV_APOSENT.map(df_func_biometricas.Nx) / bd_ativos.DT_PROV_APOSENT.map(df_func_biometricas.Dx)


df_comp_VP_ativos#.loc[30]

,Br,r-upu_t,v_r-u,ar
0,1342.21,0.151974,0.097222,9.331739
1,1302.73,0.300279,0.311805,8.812766
2,1964.27,0.164979,0.130105,8.812766
3,4599.23,0.141804,0.103056,8.812766
4,1382.88,0.162277,0.174110,8.015623
...,...,...,...,...
357,2716.30,0.206515,0.130105,10.081090
358,4046.86,0.222651,0.146186,10.081090
359,4223.40,0.183373,0.130105,9.331739
360,2511.36,0.221957,0.174110,9.331739


In [9]:
#df_func_biometricas = pd.DataFrame()

#df_func_biometricas['TESTE'] = statis_ativos.IDADE_EMT
#df_func_biometricas['TESTE1'] = statis_ativos.IDADE_EMT + 10
#df_func_biometricas['TESTE2'] = df_func_biometricas.apply(lambda x: x['TESTE']+100, axis=1)
#df_func_biometricas['x+npx'] = df_func_biometricas.TESTE1.map(lx[tabua_mortalidade]) / df_func_biometricas.TESTE.map(lx[tabua_mortalidade]) 
#df_func_biometricas

In [10]:
#𝑞𝑥(𝑚)=𝑞𝑥 ∗ [(1−0,5 ∗ 𝑖𝑥 )+(1−0,5 ∗ 𝑤𝑥 )]
#bd_ativos.DT_PROV_APOSENT